# 🏥 Nursing Function Calling with Gemini API

**No GPU required!** Uses Google's Gemini API for reliable function calling.

Gemini has **native function calling** built-in - no fine-tuning needed!

In [ ]:
!pip install -q google-generativeai

In [ ]:
import google.generativeai as genai
from google.colab import userdata

# Get API key from Colab secrets or set manually
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
except:
    GOOGLE_API_KEY = input("Enter your Google API Key: ")

genai.configure(api_key=GOOGLE_API_KEY)
print("✅ Gemini API configured!")

# List available models
print("\nAvailable models with function calling:")
for m in genai.list_models():
    if 'generateContent' in str(m.supported_generation_methods):
        print(f"  - {m.name}")

## Define Nursing Functions

These are the functions Gemini can call based on user input.

In [ ]:
# Define functions for Gemini to call
record_vitals = genai.protos.FunctionDeclaration(
    name="record_vitals",
    description="Record patient vital signs including blood pressure, heart rate, and temperature",
    parameters=genai.protos.Schema(
        type=genai.protos.Type.OBJECT,
        properties={
            "systolic": genai.protos.Schema(type=genai.protos.Type.INTEGER, description="Systolic blood pressure (top number)"),
            "diastolic": genai.protos.Schema(type=genai.protos.Type.INTEGER, description="Diastolic blood pressure (bottom number)"),
            "heart_rate": genai.protos.Schema(type=genai.protos.Type.INTEGER, description="Heart rate in beats per minute"),
            "temp_c": genai.protos.Schema(type=genai.protos.Type.NUMBER, description="Temperature in Celsius")
        }
    )
)

administer_medication = genai.protos.FunctionDeclaration(
    name="administer_medication",
    description="Log medication administration to a patient",
    parameters=genai.protos.Schema(
        type=genai.protos.Type.OBJECT,
        properties={
            "drug_name": genai.protos.Schema(type=genai.protos.Type.STRING, description="Name of the medication"),
            "dose": genai.protos.Schema(type=genai.protos.Type.STRING, description="Dose amount with units (e.g., '500mg', '1g')"),
            "route": genai.protos.Schema(type=genai.protos.Type.STRING, description="Route of administration (PO, IV, IM, SC, etc.)")
        },
        required=["drug_name", "dose", "route"]
    )
)

search_nmc_standards = genai.protos.FunctionDeclaration(
    name="search_nmc_standards",
    description="Search NMC nursing standards and guidelines",
    parameters=genai.protos.Schema(
        type=genai.protos.Type.OBJECT,
        properties={
            "query": genai.protos.Schema(type=genai.protos.Type.STRING, description="The topic to search for in NMC standards")
        },
        required=["query"]
    )
)

# Create tool with all functions
nursing_tools = genai.protos.Tool(
    function_declarations=[record_vitals, administer_medication, search_nmc_standards]
)

print("✅ Nursing functions defined!")

In [ ]:
# Create model with tools - using gemini-pro which supports function calling
model = genai.GenerativeModel(
    model_name="gemini-2.0-flash",  # Use gemini-pro if this doesn't work
    tools=[nursing_tools],
    system_instruction="""You are a clinical AI agent for nursing documentation. 
When users describe clinical observations or actions, call the appropriate function.
Extract the actual values from their input.
For medication routes: PO=oral, IV=intravenous, IM=intramuscular, SC=subcutaneous."""
)

print("✅ Model ready with nursing tools!")

In [ ]:
def process_clinical_note(note):
    """Process a clinical note and return the function call."""
    response = model.generate_content(note)
    
    # Check if a function was called
    if response.candidates[0].content.parts:
        for part in response.candidates[0].content.parts:
            if hasattr(part, 'function_call') and part.function_call:
                fc = part.function_call
                args = dict(fc.args)
                return f"{fc.name}({', '.join(f'{k}={repr(v)}' for k, v in args.items())})"
    
    return response.text if response.text else "No function call detected"

## 🧪 Test Function Calling

In [ ]:
# Test cases
test_cases = [
    "BP is 120/80, pulse 72, temp 37.2",
    "Blood pressure 145/95, heart rate 88",
    "Patient febrile at 38.5, tachycardic at 110",
    "Gave Paracetamol 1g orally",
    "Administered Morphine 5mg IV",
    "IV Flucloxacillin 1g given",
    "Insulin 10 units subcutaneous",
    "What does NMC say about confidentiality?",
    "Find NMC guidance on duty of candour",
    "Search for delegation guidelines"
]

print("🧪 Testing Gemini Function Calling")
print("="*60)

for test in test_cases:
    result = process_clinical_note(test)
    print(f"\n📝 Input: {test}")
    print(f"✅ Output: {result}")

## 📊 Run Evaluation

In [ ]:
import json
import re
from google.colab import drive

drive.mount('/content/drive')

# Load evaluation dataset
with open('/content/drive/MyDrive/nmc_brain/data/function_eval_dataset.json', 'r') as f:
    eval_data = json.load(f)

print(f"📊 Running evaluation on {len(eval_data)} test cases...")
print("="*60)

correct = 0
func_name_correct = 0
has_values = 0
results = []

for i, item in enumerate(eval_data):
    try:
        predicted = process_clinical_note(item['input'])
    except Exception as e:
        predicted = f"Error: {e}"
    
    expected = item['expected']
    
    # Check function name
    exp_func = expected.split('(')[0]
    pred_func = predicted.split('(')[0] if '(' in predicted else predicted
    if exp_func == pred_func:
        func_name_correct += 1
    
    # Check for values
    if re.search(r'=\d+', predicted) or re.search(r"='[^']+'", predicted):
        has_values += 1
    
    results.append({
        'input': item['input'],
        'expected': expected,
        'predicted': predicted,
        'func_correct': exp_func == pred_func
    })
    
    if i < 5:  # Show first 5
        status = "✅" if exp_func == pred_func else "❌"
        print(f"\n{status} Example {i+1}:")
        print(f"   Input: {item['input'][:50]}...")
        print(f"   Expected: {expected}")
        print(f"   Got: {predicted}")

n = len(eval_data)
print(f"\n" + "="*60)
print(f"📊 GEMINI API RESULTS:")
print(f"   Function Name Accuracy: {func_name_correct/n*100:.1f}%")
print(f"   Value Extraction Rate: {has_values/n*100:.1f}%")
print("="*60)

## 💾 Save for Gradio Space

This code can be used in a Hugging Face Space or any web application.

In [ ]:
# Example usage for a Gradio app
gradio_code = '''
import gradio as gr
import google.generativeai as genai
import os

# Configure API
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

# Define functions... (same as above)
# Create model... (same as above)

def process(note):
    response = model.generate_content(note)
    # Extract function call...
    return result

demo = gr.Interface(
    fn=process,
    inputs=gr.Textbox(label="Clinical Note"),
    outputs=gr.Textbox(label="Function Call"),
    title="Nursing Function Calling"
)
demo.launch()
'''

print("📝 Gradio code saved! Use this for your Space.")
print("\n💡 Tip: Set GOOGLE_API_KEY as a secret in your Hugging Face Space.")